In [1]:
# set the environment variables needed for openai package to know to reach out to azure
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://<your-endpoint.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "your AzureOpenAI key"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

from dotenv import load_dotenv
load_dotenv("../.env", override=True)

True

In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", temperature=0.5)
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)

In [3]:
from langchain.vectorstores.redis import Redis

rds = Redis.from_existing_index(
    embeddings, redis_url="redis://localhost:6379", index_name="link"
)
retriever = rds.as_retriever(search_type="similarity_limit", score_threshold=0.8)

In [4]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever)

In [5]:
from langchain.agents import Tool

tools = [
    Tool(
        name="Contract QA System",
        func=qa.run,
        description="useful for when you need to answer questions about contracts.Input needs to be in Chinese",
    )
]

In [6]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [13]:
agent_chain = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent_chain.run("在什么情况下可以单方面解除合同？")



> Entering new  chain...
Thought: This question is asking about the circumstances under which a contract can be unilaterally terminated.

Action:
```
{
  "action": "Contract QA System",
  "action_input": "在什么情况下可以单方面解除合同？"
}
```



Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists. {
  "error": {
    "message": "The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists.",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists.', 'type': 'server_error', 'param': 


Observation: 根据文本，以下情况下可以单方面解除合同：

甲方：
1. 因工负伤，治疗终结后经劳动能力鉴定机构鉴定为 1 至 4 级丧失劳动能力的；
2. 患职业病以及现有医疗条件下难以治愈的严重疾病或者精神病的；
3. 乙方正在接受纪律审查尚未作出结论的；
4. 属于国家规定的不得解除本合同的其他情形的。

乙方：
1. 在试用期内的；
2. 考入普通高等院校的；
3. 被录用或者选调为公务员的；
4. 依法服兵役的。

除上述情形外，乙方提出解除本合同未能与甲方协商一致的，乙方应当坚持正常工作，继续履行本合同；6 个月后再次提出解除本合同仍未能与甲方协商一致的，即可单方面解除本合同。

本合同可以在以下情况下单方面解除：甲方被依法注销、撤销或者解散；乙方死亡或者被人民法院宣告死亡；乙方按照国家有关规定退休或退职；甲方、乙方双方约定的合同终止条件出现。
Thought:According to the text, the circumstances under which a contract can be unilaterally terminated are:

For Party A:
1. Suffering from work-related injuries and being appraised by a labor capacity appraisal institution as having lost 1 to 4 levels of labor capacity after the end of treatment;
2. Suffering from occupational diseases or serious diseases or mental illnesses that are difficult to cure under existing medical conditions;
3. Being under disciplinary investigation by Party B without a conclusion yet;
4. Other circumstances stipulated by the state that cannot terminate the contract.

For Pa

'The circumstances under which a contract can be unilaterally terminated are listed above for both Party A and Party B.'

In [14]:
from langchain import OpenAI, ConversationChain
from langchain.schema import BaseMemory
from pydantic import BaseModel
from typing import List, Dict, Any

In [1]:
class DocsMemory(BaseMemory, BaseModel):
    """Memory class for storing information about entities."""

    memories: Dict[str, Any] = dict()

    def clear(self):
        self.entities = {}

    @property
    def memory_variables(self) -> List[str]:
        """Define the variables we are providing to the prompt."""
        return list(self.memories.keys())

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Load the memory variables, in this case the entity key."""
        return self.memories

    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """Save context from this conversation to buffer."""
        print("Saving context...")
        print(outputs)

NameError: name 'BaseMemory' is not defined

In [18]:
qa_test = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    memory=DocsMemory(),
    return_source_documents=True,
)

In [19]:
qa_test("在什么情况下可以单方面解除合同？")

Saving context...
{'query': '在什么情况下可以单方面解除合同？'}


{'query': '在什么情况下可以单方面解除合同？',
 'result': '根据文本，以下情况可以单方面解除合同：\n\n- 在承担国家重大科研项目期间；\n- 掌握重大科技成果关键技术和资料，未脱离保密期；\n- 被审查期间或因经济问题未作结案处理之前；\n- 合同订立时所依据的客观情况发生重大变化，致使合同无法履行，经甲乙双方协商不能就变更合同达成协议的；\n- 甲方提出解除本合同，乙方同意解除的；\n- 乙方患病或者非因工负伤，医疗期满后，不能从事原工作也不能从事由甲方安排的其他工作，甲方单方面解除本合同的；\n- 乙方年度考核或者聘期考核不合格，又不同意甲方调整其工作岗位的；\n- 在试用期内被证明不符合本岗位要求；\n- 连续旷工超过 10 个工作日或者 1 年内累计旷工超过 20 个工作日的；\n- 乙方公派出国或因私出境逾期不归；\n- 违反工作规定或者操作规程，发生责任事故，或者失职、渎职，造成严重后果的；\n- 严重扰乱工作秩序，致使甲方、其他单位工作不能正常进行的；\n- 被判处拘役、有期徒刑缓刑以及有期徒刑以上刑罚收监执行，或者被劳动教养的；\n- 乙方患病或者非因工负伤，医疗期满后，不能从事原工作也不能从事由甲方安排的其他工作的。\n\n此外，根据文本，还有以下情况可以单方面解除合同：\n\n- 在试用期内的；\n- 考入普通高等院校的；\n- 被录用或者选调为公务员的；\n- 依法服兵役的；\n- 乙方按照国家有关规定退休或退职的；\n- 乙方死亡或者被人民法院宣告死亡的；\n- 甲方被依法注销、撤销或者解散的。\n\n合同期限届满和甲、乙双方约定的合同终止条件出现也可以导致合同终止。',
 'source_documents': [Document(page_content='（六）有下列情形之一的，乙方不得单方面解除项目聘用合同： 1．在承担国家重大科研项目期间； 2．掌握重大科技成果关键技术和资料，未脱离保密期； 3．被审查期间或因经济问题未作结案处理之前。 （七）本合同订立时所依据的客观情况发 生重大变化，致使合同无法履行，\n经甲乙双方协商不能就变更合同达成协议的，双方均可以单方面解除本合同。 \n（八）有下列情形之一的，甲方应当根据乙方在本单位的实际工作年限向其\n支付经济补偿： \n1．甲方提出解除本合同，乙方同意解除